**Connect google drive**

In [1]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Fri Aug 19 06:48:40 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive/

Mounted at /content/gdrive
'Colab Notebooks'   yolo


**1) Clone the Darknet**



In [3]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15490, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 15490 (delta 10), reused 18 (delta 8), pack-reused 15460
Receiving objects: 100% (15490/15490), 14.08 MiB | 9.56 MiB/s, done.
Resolving deltas: 100% (10398/10398), done.


**2) Compile Darknet using Nvidia GPU**


In [4]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

**3) Configure Darknet network for training YOLO V3**

In [6]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [7]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [8]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolo/yolov3"

mkdir: cannot create directory ‘/mydrive/yolo/yolov3’: File exists


In [9]:
!echo "cercospora" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolo/yolov3' > data/obj.data
!mkdir data/obj

In [10]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2022-08-19 06:52:21--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  17.0MB/s    in 10s     

2022-08-19 06:52:32 (15.0 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [13]:
!unzip /mydrive/yolo/yolov3/plant_diseases_2.zip -d data/obj

Archive:  /mydrive/yolo/yolov3/plant_diseases_2.zip
  inflating: data/obj/image_.jpg     
  inflating: data/obj/image_.txt     
  inflating: data/obj/image_1.jpg    
  inflating: data/obj/image_1.txt    
  inflating: data/obj/image_10.jpg   
  inflating: data/obj/image_10.txt   
  inflating: data/obj/image_11.jpg   
  inflating: data/obj/image_12.jpg   
  inflating: data/obj/image_12.txt   
  inflating: data/obj/image_13.jpg   
  inflating: data/obj/image_13.txt   
  inflating: data/obj/image_14.jpg   
  inflating: data/obj/image_14.txt   
  inflating: data/obj/image_15.jpg   
  inflating: data/obj/image_15.txt   
  inflating: data/obj/image_16.jpg   
  inflating: data/obj/image_16.txt   
  inflating: data/obj/image_17.jpg   
  inflating: data/obj/image_17.txt   
  inflating: data/obj/image_18.jpg   
  inflating: data/obj/image_18.txt   
  inflating: data/obj/image_19.jpg   
  inflating: data/obj/image_19.txt   
  inflating: data/obj/image_2.jpg    
  inflating: data/obj/image_20.jpg  

In [14]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

0 0.497500 0.508443 0.972500 0.983114

['0', '0.497500', '0.508443', '0.972500', '0.983114']
0 data/obj/image_13.txt
0 0.497500 0.508443 0.972500 0.983114
0 0.245614 0.483051 0.400000 0.932203

['0', '0.245614', '0.483051', '0.400000', '0.932203']
1 data/obj/image_44.txt
0 0.245614 0.483051 0.400000 0.932203
0 0.805263 0.488701 0.298246 0.638418

['0', '0.805263', '0.488701', '0.298246', '0.638418']
1 data/obj/image_44.txt
0 0.805263 0.488701 0.298246 0.638418
0 0.512891 0.483594 0.725781 0.896875

['0', '0.512891', '0.483594', '0.725781', '0.896875']
2 data/obj/image_8.txt
0 0.512891 0.483594 0.725781 0.896875
0 0.516406 0.506944 0.861979 0.986111

['0', '0.516406', '0.506944', '0.861979', '0.986111']
3 data/obj/image_16.txt
0 0.516406 0.506944 0.861979 0.986111
0 0.507722 0.500000 0.984556 0.958763

['0', '0.507722', '0.500000', '0.984556', '0.958763']
4 data/obj/image_36.txt
0 0.507722 0.500000 0.984556 0.958763
0 0.500000 0.497512 0.930556 0.985075

['0', '0.500000', '0.497512', '0

In [15]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/image_51.jpg', 'data/obj/image_5.jpg', 'data/obj/image_25.jpg', 'data/obj/image_60.jpg', 'data/obj/image_65.jpg', 'data/obj/image_63.jpg', 'data/obj/image_26.jpg', 'data/obj/image_8.jpg', 'data/obj/image_6.jpg', 'data/obj/image_1.jpg', 'data/obj/image_62.jpg', 'data/obj/image_17.jpg', 'data/obj/image_13.jpg', 'data/obj/image_33.jpg', 'data/obj/image_30.jpg', 'data/obj/image_52.jpg', 'data/obj/image_46.jpg', 'data/obj/image_10.jpg', 'data/obj/image_7.jpg', 'data/obj/image_15.jpg', 'data/obj/image_9.jpg', 'data/obj/image_23.jpg', 'data/obj/image_59.jpg', 'data/obj/image_53.jpg', 'data/obj/image_48.jpg', 'data/obj/image_55.jpg', 'data/obj/image_22.jpg', 'data/obj/image_32.jpg', 'data/obj/image_12.jpg', 'data/obj/image_37.jpg', 'data/obj/image_47.jpg', 'data/obj/image_50.jpg', 'data/obj/image_38.jpg', 'data/obj/image_36.jpg', 'data/obj/image_40.jpg', 'data/obj/image_21.jpg', 'data/obj/image_4.jpg', 'data/obj/image_34.jpg', 'data/obj/image_35.jpg', 'data/obj/image_58.jpg', 'data/

In [16]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Görüntülenen çıkış son 5000 satıra kısaltıldı.
 total_bbox = 277103, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.905597), count: 5, class_loss = 0.004726, iou_loss = 0.074055, total_loss = 0.078780 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.743369), count: 1, class_loss = 0.222336, iou_loss = 0.078968, total_loss = 0.301304 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000100, iou_loss = 0.000000, total_loss = 0.000100 
 total_bbox = 277109, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.890274), count: 4, class_loss = 0.000001, iou_loss = 0.082880, total_loss = 0.082881 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (mse lo